# NLP Lab : Modèles de langue

Dans ce tp, nous allons constuire les briques principales du modèle GPT2 et entrainer un petit modèle sur des poèmes de Victor Hugo. 

Les questions sont posées dans ce notebook, mais pour executer l'entrainement, il faudra modifier le ficher gpt_single_head.py


## Données

Les données d'entrainement sont un recueil de poèmes de Victor Hugo issu du site [gutenber.org](https://www.gutenberg.org/). 

Afin de réduire la complexité du modèle, nous allons modéliser le texte au niveau caractère. 

Questions:
>* en utilisant [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter), afficher le nombre de caractères différents dans le texte et la fréquence de chaque caractère.

In [174]:
import collections

with open('data/hugo_contemplations.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'Number of characters in the file: {len(text)}')
# here are all the unique characters that occur in this text
counter = collections.Counter(text)
chars = counter.keys()
vocab_size = len(chars)

print (f'Number of character in counter: {sum(counter.values())}')
print (f'{len(chars)} different characters')
print (counter)


Number of characters in the file: 285222
Number of character in counter: 285222
101 different characters
Counter({' ': 49127, 'e': 30253, 's': 17987, 'u': 14254, 'r': 14223, 't': 14071, 'a': 14048, 'n': 13725, 'i': 12828, 'o': 12653, 'l': 11638, '\n': 8102, 'm': 6495, 'd': 6375, ',': 6077, 'c': 5074, 'p': 4206, "'": 3820, 'v': 3492, 'é': 2943, 'b': 2783, 'f': 2772, 'h': 2221, 'q': 1956, 'g': 1790, '.': 1420, 'x': 1154, 'L': 1147, '!': 1121, 'E': 1074, ';': 1043, '-': 1020, 'j': 890, 'D': 764, 'è': 725, 'à': 706, 'y': 660, 'I': 627, 'ê': 605, 'C': 593, 'S': 545, 'A': 530, 'Q': 503, 'z': 482, 'J': 471, 'O': 450, 'T': 441, 'P': 435, '?': 388, 'V': 383, 'â': 381, 'N': 362, 'M': 344, 'ù': 298, ':': 294, 'R': 240, 'î': 214, 'U': 208, 'ô': 159, 'X': 150, '1': 146, 'H': 116, 'F': 114, '5': 111, '8': 93, 'B': 78, '«': 74, 'É': 70, '»': 69, 'G': 67, '4': 64, 'û': 62, '3': 47, 'ç': 34, 'À': 33, 'ë': 32, 'ï': 31, '2': 30, '·': 26, 'Ê': 24, '6': 23, '7': 23, 'Ô': 19, '9': 19, 'È': 11, 'k': 10, '0':

### Encodage / décodage
Afin de transformer le texte en vecteur pour le réseau de neurone, il faut encoder chaque caractère avec un entier. Les fonctions suivante opérent l'encodage et le décodage des caractères:

In [175]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: transform a string into a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: transform a list of integers into a string


# test that your encoder/decoder is coherent
testString = "\nDemain, dès l'aube"
assert decode(encode (testString)) ==  testString

### Découpage Train/Validation

L'objectif étant de prédire des poèmes, il ne faut pas mélanger les lignes aléatoirements. Il faut garder l'ordre des lignes dans le texte et uniquement prendre les premier 90% pour entrainer et les 10% restant pour contrôler l'apprentissage. 

In [180]:
import torch
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
## YOUR CODE HERE
# first 90% will be train, rest val
n = int(0.9*len(data))
###
train_data = data[:n]
val_data = data[n:]

### Contexte

Le modèle de langue possède comme paramètre la taille maximale du contexte à considérer pour faire la prédiction du prochain caractère. Ce contexte est appelé `block_size`. Les données d'apprentissage sont donc des sequences de charactères consécutifs, issues de l'ensemble d'entraînenement tirées aléatoirement et de longueur `block_size`.

Si le caractère de début de la séquence est `i`, la séquence de contexte est donc :
``` x = data[i:i+block_size]```
et la valeur à prédire à chaque position dans le contexte est le caractère suivant:
```y  = [data[i+1:i+block_size+1]




In [181]:
block_size = 8

i  = torch.randint(len(data) - block_size, (1,))
print (i)
x = train_data[i:i+block_size]
y = train_data[i+1:i+1+block_size]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print (f'context is >{decode(context.tolist())}< target is >{decode([target.tolist()])}<')

tensor([21574])
context is >r< target is >e<
context is >re< target is > <
context is >re < target is >v<
context is >re v< target is >o<
context is >re vo< target is >l<
context is >re vol< target is >o<
context is >re volo< target is >n<
context is >re volon< target is >t<


### Définition des batchs

In [182]:
batch_size = 4
torch.manual_seed(2023)
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # select batch_size starting points in the data, store them in a list called starting_points
    starting_points = torch.randint(len(data) - block_size, (batch_size,))
    # x is the sequence of integer starting at each straing point and of length block_size
    x = torch.stack([data[i:i+block_size] for i in starting_points])
    # y is the character after each starting position
    y = torch.stack([data[i+1:i+block_size+1] for i in starting_points])
    # send data and target to device
    x, y = x.to(device), y.to(device)
    return x, y

### Premier modèle: un bigramme 

Prédit le caractère suivant uniquement en fonction du caractère courant.
Optimisé par descente de gradient

In [183]:
import torch.nn as nn

# use a gpu if we have one
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # we use a simple vocab_size times vocab_size tensor to store the probabilities 
        # of each token given a single token as context
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (Batch,Time) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch,Time,Channel)
   
        # don't compute loss if we don't have targets
        if targets is None:
            loss = None
        else:
            # change the shape of the logits and target to match what is needed for CrossEntropyLoss
            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            Batch, Time, Channels = logits.shape
            logits = logits.view(Batch*Time, Channels)
            targets = targets.view(Batch*Time)
            
            # negative log likelihood between prediction and target
            loss = nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
# send the model to device
m = model.to(device)

Les poids étant unitialisés avec une distribution normale N(0,1) sur chaque dimension, la loss attendue après l'initialisation est `-ln(1/vocab_size)`

In [184]:
xb, yb = get_batch('train')
logits, loss = m(xb, yb)
print (logits.shape)
print (loss)

torch.Size([32, 101])
tensor(5.0953, grad_fn=<NllLossBackward0>)


In [185]:
print (out.shape)

torch.Size([4, 8, 16])


In [186]:
print (encode(['\n']))
print (idx)
prompt = torch.ones((1,1), dtype=torch.long, device=device)*6
print (decode(m.generate(prompt,max_new_tokens=100)[0].tolist()))

[3]
tensor([[3]])
N)G(F(ÎG7

;Ww«ïIKK[G9bÀB;7ÔZU5pWt3?îzê
Zû7àâËZ!5-F[HëK«mxMLWÉn8ÆSê_Rïcx.7P)(Ca5î.d-eZEn,jfJF8,U3ÂdOè


### Entrainement

In [190]:
max_iters = 100
batch_size = 32
eval_interval = 20
learning_rate = 1e-3
eval_iters = 20

@torch.no_grad() # no gradient is computed here
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 2.3970, val loss 2.4571
step 20: train loss 2.4051, val loss 2.4683
step 40: train loss 2.4035, val loss 2.4442
step 60: train loss 2.4001, val loss 2.4668
step 80: train loss 2.3817, val loss 2.4844


In [191]:
idx = torch.ones((1,1), dtype=torch.long)*3
print (decode(m.generate(idx,max_new_tokens=100)[0].tolist()))



Hécen pr,  drmmou des lent d, êt'he le pit e, mes sple ls fêmafans.
 quie-t  pait, e, t rt   d  yrp


## Single Head Attention



![single head attention](images/single_head_attention.png)


Le futur n'est pas utilisé pour prédire (le futur).

In [192]:
T = 8

# first version of the contraints with matrix multiplication
weights0 = torch.tril(torch.ones(T,T))
weights0 = weights / weights.sum(1, keepdim=True) 
print (weights0)

tensor([[[4.8368e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.0541e-01, 2.6684e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.9503e-01, 1.4916e-01, 4.8819e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.9977e-02, 3.8200e-01, 1.0825e-01, 3.8721e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.2269e-02, 3.6319e-02, 1.6390e-01, 1.1026e-02, 5.1437e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [4.7134e-03, 9.5730e-02, 9.7573e-02, 3.9572e-01, 1.1593e-01,
          4.6665e-02, 0.0000e+00, 0.0000e+00],
         [3.9678e-02, 6.1178e-02, 4.8630e-02, 2.1766e-02, 3.5506e-01,
          9.2121e-01, 5.7621e-01, 0.0000e+00],
         [2.9240e-02, 8.7719e-03, 9.3462e-02, 5.3277e-01, 1.4638e-02,
          3.2123e-02, 4.2379e-01, 1.0000e+00]],

        [[2.7494e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

In [149]:
tril = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril== 0, float('-inf'))
weights = nn.functional.softmax(weights, dim=-1)
print (weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


Attention head

* créer les couches key, query et value comme des couches linéaires de dimension C x head_size
* appliquer les couches à x
* weights = query x key (transposer les deuxième et troisième dimensions de key pour pouvoir faire le produit)
* appliquer le facteur de normalisation 
* appliquer le masque triangulaire  et la softmax à weight
* appliquer value à x
* le résultat `out` est la multiplication de weights par value(x)


In [159]:
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
key = nn.Linear(C, head_size, bias=False)
query =  nn.Linear(C, head_size, bias=False)
value =  nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
v = value(x) # (B, T, head_size)
weights = q @ k.transpose(1, 2) * head_size**-0.5 # (B, T, head_size) @ (B, 16, head_size) -> (B, T, T)
weights = weights.masked_fill(tril== 0, float('-inf'))
weights = nn.functional.softmax(weights, dim=-1)
out  = weights @ value(x) # (B, T, head_size)

In [161]:
weights[0]
out[0]

tensor([[-4.8218e-01, -1.6254e-01,  4.6861e-02, -1.1899e-01, -4.8324e-01,
         -5.2388e-01,  6.5677e-03, -5.3459e-02,  2.7850e-01, -4.1425e-01,
         -3.3740e-01,  1.5587e-01, -2.5850e-01,  2.1828e-01,  7.7525e-03,
          3.7400e-01],
        [ 5.8942e-01,  1.1879e-02,  3.4260e-01,  4.2746e-01, -4.6086e-01,
         -4.5144e-01, -4.6541e-01, -1.6498e-01,  3.2426e-01, -5.3940e-02,
          1.3195e-02,  5.8804e-02, -4.8058e-01,  5.2046e-01,  3.5556e-01,
          1.9390e-01],
        [ 2.5004e-01,  1.2970e-02, -7.3553e-02,  4.0934e-01, -3.9027e-01,
         -3.6913e-01, -2.9485e-01, -9.4224e-02,  2.2517e-02, -1.4430e-01,
          1.1435e-01,  5.0670e-02, -5.0698e-01,  4.4101e-01,  2.7340e-01,
          1.8484e-01],
        [ 1.0507e+00,  9.5079e-02,  4.1365e-01,  7.0172e-01, -3.9971e-01,
         -3.4922e-01, -6.4075e-01, -2.0988e-01,  2.2971e-01,  1.1620e-01,
          2.4551e-01,  6.4572e-04, -5.7112e-01,  6.7137e-01,  5.3380e-01,
          7.7160e-02],
        [-1.9791e-01


Questions:

> * Copier votre code dans `gpt_single_head.py` et faite un entrainement.
> * Quelle loss en train et val obtenez vous ? Le texte vous parait-il meilleur ?

step 4999: train loss 2.3324, val loss 2.4422

## Multi-head attention

La *multi-head attention* est simplement le calcul en parallèle de plusieurs *single head attention*. Chacun des single head attention est concaténée pour créer la sortie de la multi-head attention. Dans la figure issue de l'article original, le nombre de *heads* dans le *multi-head* est `h`. Afin d'opérer des combinaisons pondées sur la sortie de chacune des single head, une couche de calcul linéaire est ajoutée.

![multi head attention](images/multi_head_attention.png)

Le code ci-dessous crée un module de multi-head attention.
Question:
> * dans le constructeur, créer une liste contenant `num_heads` module `Head` en utilisant la fonction [ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) de pytorch 
> * dans la fonction `forward`, appliquer chaque single head à l'input et concaténer le résultat en utilisant la fonction [cat](https://pytorch.org/docs/stable/generated/torch.cat.html) de pytorch

In [163]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        ## YOUR CODE HERE
        ## list of num_heads modules of type Head
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        ###
        
    def forward(self, x):
        ## YOUR CODE HERE
        ## apply each head in self.heads to x and concat the results 
        out = torch.cat([h(x) for h in self.heads], dim=-1)

        return out


Questions:
> * copier le fichier gpt_single_head.py en gpt_multi_head.py
> * ajouter le module MultiHeadAttention dans gpt_multi_head.py
> * en tête de fichier, ajouter un paramètre  `n_head = 4`
> * dans le module BigramLanguageModel, remplacer le module Head par un module MultiHeadAttention avec la paramètres `num_heads = n_head` et `head_size = n_embd // n_head` pour garder le même nombre de paramètres.
> relancer l'entrainement et noter le nombre de paramètres et les loss obtenues



In [ ]:
0.009893 M parameters
step 4999: train loss 2.1570, val loss 2.1802

## Ajout d'une couche de calcul FeedForward


Après les couches d'attention qui collectent l'information dans la séquence, une couche de calcul est ajoutée pour combiner toutes les informations de la séquence. Cette couche est un simple Multi-Layer-Perceptron avec une couche cachée et une non linéarité de type [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
![multi feedfoward](images/multi_ff.png)

In [167]:
class FeedForward(nn.Module):
    """ a simple MLP with RELU """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

Question
> * ajouter le module `FeedForward` dans gpt_multi_head.py
> * ajouter cette couche `FeedForward` après la *multi-head attention*
> * relancer l'entrainement et noter le nombre de paramètres et les loss obtenues

0.010949 M parameters
step 4999: train loss 2.1290, val loss 2.1216

## Empiler les blocs

Le réseau construit jusqu'à présent n'est en fait qu'un bloc du réseau final. Il est maintenant possible d'empiler les blocs de *multi-head attention* pour créer un réseau profond. 

![multi feedfoward](images/multi_bloc.png)


Le code suivant crée un bloc : 

In [168]:
class Block(nn.Module):
    """ A single bloc of multi-head attention """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x

Question
> * ajouter le module `Block` dans gpt_multi_head.py
> * modifier le code de `BigramLanguageModel` pour ajouter 3 `Block(n_embd, n_head=4)` avec un contaier [Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) à la place de `MultiHeadAttention`et `FeedForward`
> * relancer l'entrainement et noter le nombre de paramètres et les loss obtenues

In [ ]:
0.019205 M parameters
step 4999: train loss 2.2080, val loss 2.2213

## Amélioration de l'entraînement

Si on veut continuer à augmenter la taille du réseau, il est nécessaire d'utiliser des couches permettant d'améliorer l'entraînement et ses capacités de généralisation (réduire le sur-apprentissage). Ces couches sont :
- *skip connections* ou *residual connections*
- les couches de normalisation
- le dropout


![multi feedfoward](images/multi_skip_norm.png)


Questions:
> * dans le module Bloc, ajouter une skip connection en ajoutant l'input dans chaque connexion: 
```
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
```
> * dans le module Bloc, ajouter 2 couches de [LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) de taille `n_embd` avant la couche de *Multi-Head attention* et avant la *FeedForward*
> * après la série de 3 blocs, ajouter une couche de LayerNorm de taille `n_embd` 
> * définir une variable `dropout = 0.2` en début de fichier et ajouter une couche de [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)
>    * après la couche RELU dans FeedForward
>    * après la couche de MultiHead dans `MultiHeadAttention`
>    * après la softmax dans la single head attention `Head`
> * relancer l'entrainement et noter le nombre de paramètres et les loss obtenues


In [ ]:
0.019653 M parameters

## Conclusion

Les principaux éléments de GPT2 sont en place, il faut maintenant faire passer le modèle à l'échelle et l'entraîner sur une base de données beaucoup plus grande. Pour comparaison, les paramètres de [GPT2](https://huggingface.co/transformers/v2.11.0/model_doc/gpt2.html) sont : 

* `vocab_size = 50257` : GPT2 modélise des token (subword) alors que nous modélisons des caractères. Pour nous, `vocab_size = 100`
* `n_positions = 1024` : la taille maximale du contexte. Pour nous, c'est `block_size = 8`
* `n_embd = 768`:  la dimension des embeddings. Pour nous c'est `n_embd = 32`
* `n_layer = 12`: le nombre de block. Pour nous c'est 3.
* `n_head = 12`: le nombre de multi-head attention. Pour nous c'est 4.

Au total, GPT2 a 1500 millions de paramètres et a été entrainé sur 8M de pages web, soit 40 Gb de texte.
